# Problem-1 (Question-1) How To Create Dataframes From Wikipedia Tables

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Use SHIFT+TAB keys to popup inplace code help
%config IPCompleter.greedy = True

# Output multiple statements from one input cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Customize Notebook

**table_from_top.** If the Wikipedia page has one table then use `table_from_top = 1` value. Otherwise count table number from top and replace value to get specific table.

**wikipedia_page.** Specify the wikipedia page name from where to source dataset. The CSV file will be saved with the same name.

**trace.** Set `trace = True` to trace how feature values are extracted. Does not save extracted dataset. Prefixes applied parsing/extraction rules to extracted values.

In [2]:
table_from_top = 1
wikipedia_page = 'List_of_postal_codes_of_Canada:_M'
trace = False

## Load and Parse

This section loads the Wikipedia page and parses the table data we are interested in converting to a dataset.

In [3]:
wikipedia_url = 'https://en.wikipedia.org/wiki/{}'.format(wikipedia_page)
page = requests.get(wikipedia_url)
soup = BeautifulSoup(page.content, 'lxml')
tables = soup.find_all('table', {'class': 'wikitable'})
table = tables[table_from_top - 1]

## Quick Preview

This section extracts the table header with feature or column names.

Use this section to quick preview if you have the right table in processing.

In [4]:
feature_names = []

header_row = table.find('tr')
for header in header_row.find_all('th'):
    feature_name = ' '.join(header.find_all(text=True))    
    feature_names.append(feature_name.replace('\n', ''))
print(feature_names)

['Postcode', 'Borough', 'Neighbourhood']


## Data Wrangling

This section applies data wrangling rules based on exceptions found when parsing Wikipedia tables.

- If a feature value contains a link then extract text from the link.
- Ignore text which starts with `[` square brackets.
- Ignore image links (...flags) prefix link text.
- Ignore hidden text used for IDs.

In [5]:
samples = []
sample_rows = table.find_all('tr')[1:]
for sample_row in sample_rows:
    features = []
    for feature_col in sample_row.find_all('td'):
        feature_value = ''
        text = feature_col.text.replace('\n','')
        features.append(text)
        href_tags = soup.find_all(href=True)
    features.append(feature_value)
    samples.append(dict(zip(feature_names, features)))
#sample_rows

## Preview Dataset

This section enables you to preview the parsed dataset.

In [6]:
df = pd.DataFrame(samples)
#col_list = list(df)
# use this handy way to swap the elements
#col_list[0], col_list[1],  col_list[2] = col_list[2], col_list[0], col_list[1]
# assign back, the order will now be swapped
#df.columns = col_list
final_df = df[["Postcode", "Borough", "Neighbourhood"]]
#final_df
final_df.head()
final_df.tail()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


,Postcode,Borough,Neighbourhood
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor
287,M9Z,Not assigned,Not assigned


# Output Data Frame Creation
### Group the Neighbourhood based on the Postal Code

In [7]:
#Group the Neighbourhood based on the Postal Code
output_df = final_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
ordered_df = output_df[['Postcode','Borough', 'Neighbourhood']]

# Assign the Borough value to Neighbourhood value if the Borough has a value and the Neighbourhood has no assigned value

In [8]:
for i, row in ordered_df.iterrows():
    if(row['Borough'] != 'Not assigned' and row['Neighbourhood'] == 'Not assigned'):
        print(row['Borough'])
        row['Neighbourhood'] = row['Borough']

Queen's Park


# Display the final Output

In [9]:
#for i, row in ordered_df.iterrows():
#    print(row['Postcode'], row['Borough'], row['Neighbourhood'])
ordered_df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
8,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
9,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"


# Display the shape of the Output

In [10]:
ordered_df.shape

(180, 3)

# Problem-2(Question-2)Read the Co-Ordinates of Canadian Geo Spatial Data into a Data Frame

In [11]:
coordinates_df = pd.read_csv("https://cocl.us/Geospatial_data")
coordinates_df = coordinates_df.rename(columns={'Postal Code': 'Postcode'})
#coordinates_df.head()
coordinates_df.shape

(103, 3)

# Merge the Neighbourhood Data with the Geo Spatial Data using Postal Code

In [12]:
#combined_df = pd.concat([ordered_df,coordinates_df], axis=0, ignore_index=True)
combined_df = pd.merge(ordered_df, coordinates_df, on='Postcode')
combined_df
#for i, row in combined_df.iterrows():
#    print(row['Postcode'], row['Borough'], row['Neighbourhood'],row['Latitude'],row['Longitude'])
#for i, row in combined_df.iterrows():
#    if("Toronto" in row['Borough']):
#        print(i,row['Borough'])
#print(row['Postcode'], row['Borough'], row['Neighbourhood'],row['Latitude'],row['Longitude'])

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [13]:
print('The dataframe has {} Boroughs and {} Neighborhoods.'.format(
        len(combined_df['Borough'].unique()),
        combined_df.shape[0]
    )
)

The dataframe has 11 Boroughs and 103 Neighborhoods.
